In [1]:
import json
import geopandas as gp
from shapely.geometry import Point, LineString, Polygon

In [2]:
# these are the unedited data from Ollie O'Brien's vis repo (https://github.com/oobrien/vis)
with open("static/tfl_lines.geojson", 'r') as f:            
    gj = json.loads(f.read()) 

In [3]:
# Some segments belong to multiple lines
# We need to build new LineStrings with each of these
# this results in a small amount of duplication but that's OK
new_features = []
for feature in gj['features']:
    num_lines = len(feature['properties']['lines'])
    if num_lines > 1:
        for segment in feature['properties']['lines'][1:]:
            new_feature = feature
            new_feature['properties']['line_name'] = segment['name']
            new_feature['properties']['shared'] = True
            new_features.append(
                new_feature
            )
    else:
        feature['properties']['shared'] = False

In [4]:
# assign a new line_name property to existing features
for feature in gj['features']:                              
    feature['properties']['line_name'] = feature['properties']['lines'][0]['name']

In [5]:
# combine existing and new features
gj['features'] = gj['features'] + new_features

In [6]:
# remove 'lines' property, since we don't need it any more
for feature in gj['features']:
    try:
        del feature['properties']['lines']
    except KeyError:
        pass

In [7]:
with open("static/tfl_lines_edited.geojson", 'w') as g:
    json.dump(gj, g)

In [13]:
df = gp.read_file("static/tfl_lines_edited.geojson")

In [14]:
df.head()

,shared,id,line_name,geometry
0,0,LeaValleyLine1,London Overground,LINESTRING (-0.059409291146068 51.523912138223...
1,0,LeaValleyLine2,London Overground,LINESTRING (-0.057597366770253 51.532454060856...
2,0,LeaValleyLine3,London Overground,LINESTRING (-0.05785044994051 51.5411764951798...
3,0,LeaValleyLine4,London Overground,LINESTRING (-0.061160397519048 51.549089532848...
4,0,LeaValleyLine5,London Overground,LINESTRING (-0.06845384240135299 51.5586559609...


In [6]:
# Mapbox GL can only extrude polygons.
# LineStrings can be converted into Polygons by buffering them
# choose an appropriate width here
df.geometry = df.geometry.buffer(0.000100)

In [7]:
# fix up other unfamiliar lines
df['line_name'] = df['line_name'].replace('East London', 'London Overground')
df['line_name'] = df['line_name'].replace('Crossrail 2', 'Crossrail')
df['line_name'] = df['line_name'].replace('TfL Rail', 'Crossrail')

In [10]:
df.line_name.unique()

array([u'London Overground', u'Victoria', u'Piccadilly', u'District',
       u'Circle', u'Hammersmith & City', u'Metropolitan', u'Central',
       u'Jubilee', u'DLR', u'Bakerloo', u'Northern', u'Waterloo & City',
       'Crossrail', u'Tramlink', u'Emirates Air Line'], dtype=object)


In [36]:
df.to_file("static/tube_polygons.geojson", driver="GeoJSON")

In [15]:
!jupyter labextension install @jupyterlab/git

> /usr/local/bin/npm pack @jupyterlab/git
npm notice 
npm notice 📦  @jupyterlab/git@0.5.0
npm notice === Tarball Contents === 
npm notice 2.1kB  package.json                                      
npm notice 1.5kB  LICENSE                                           
npm notice 2.6kB  README.md                                         
npm notice 1.2kB  lib/components/BranchHeader.d.ts                  
npm notice 8.3kB  lib/components/BranchHeader.js                    
npm notice 741B   lib/components/CommitBox.d.ts                     
npm notice 2.4kB  lib/components/CommitBox.js                       
npm notice 1.6kB  lib/components/FileItem.d.ts                      
npm notice 7.8kB  lib/components/FileItem.js                        
npm notice 3.8kB  lib/components/FileList.d.ts                      
npm notice 21.0kB lib/components/FileList.js                        
npm notice 1.1kB  lib/components/GitPanel.d.ts                      
npm notice 10.1kB lib/components/GitPanel.js 